In [19]:
import random
import time
import csv
import math

class MCTSTree:
    def __init__(self, move, parent=None):
        self.move = move
        self.parent = parent
        self.children = []
        self.visits = 0
        self.score = 0

    def add_child(self, move):
        child = MCTSTree(move, self)
        self.children.append(child)
        return child

class HexGame:
    def __init__(self, size):
        self.size = size
        self.board = [['-' for _ in range(size)] for _ in range(size)]
        self.total_search_space = 0  # Initialize total_search_space
        self.total_elapsed_time = 0

    def print_board(self):
        for row in self.board:
            print(' '.join(row))
        print()

    def is_valid_move(self, move):
        x, y = move
        return 0 <= x < self.size and 0 <= y < self.size and self.board[x][y] == '-'

    def is_winner(self, player):
        # Check if there's a winning path from top to bottom (for 'X') or from left to right (for '#')
        if player == '#':
            return any(self.board[0][j] == '#' and self.dfs((0, j), '#', set()) for j in range(self.size))
        else:
            return any(self.board[i][0] == 'X' and self.dfs((i, 0), 'X', set()) for i in range(self.size))

    def dfs(self, pos, player, visited):
        if pos in visited or not self.is_valid_move(pos) or self.board[pos[0]][pos[1]] != player:
            return False
        if (player == '#' and pos[0] == self.size - 1) or (player == 'X' and pos[1] == self.size - 1):
            return True

        visited.add(pos)
        neighbors = [(pos[0] + 1, pos[1]), (pos[0] - 1, pos[1]), (pos[0], pos[1] + 1), (pos[0], pos[1] - 1)]
        return any(self.dfs(n, player, visited.copy()) for n in neighbors)

    def get_empty_cells(self):
        return [(i, j) for i in range(self.size) for j in range(self.size) if self.board[i][j] == '-']

    def is_game_over(self):
        return self.is_winner('#') or self.is_winner('X') or len(self.get_empty_cells()) == 0

    def ucb1(self, node):
        if node.visits == 0:
            return float('inf')
        return node.score / node.visits + math.sqrt(2 * math.log(node.parent.visits) / node.visits)

    def select_node(self, node):
        while node.children:
            node = max(node.children, key=lambda x: self.ucb1(x))
        return node

    def expand_node(self, node):
        move = random.choice(self.get_empty_cells())
        return node.add_child(move)

    def simulate_game(self, node):
        sim_board = [row[:] for row in self.board]
        player = 'X'
        empty_cells = self.get_empty_cells()
        random.shuffle(empty_cells)  # Shuffle the empty cells to introduce randomness
        for move in empty_cells:
            sim_board[move[0]][move[1]] = player
            if player == 'X':
                player = '#'
            else:
                player = 'X'
            if self.is_winner('#') or self.is_winner('X') or len(self.get_empty_cells()) == 0:
                break
        winner = '#' if self.is_winner('#') else ('X' if self.is_winner('X') else None)
        return winner


    def backpropagate(self, node, winner):
        while node is not None:
            node.visits += 1
            if winner == 'X':
                node.score += 1
            elif winner == '#':
                node.score -= 1
            node = node.parent
            self.total_search_space += 1  # Increment total_search_space
    def get_best_move(self):
        root = MCTSTree(None)
        for _ in range(1000):  # Number of simulations
            node = root
            # Selection
            node = self.select_node(node)
            # Expansion
            if not self.is_game_over():
                node = self.expand_node(node)
            # Simulation
            winner = self.simulate_game(node)
            # Backpropagation
            self.backpropagate(node, winner)

        best_move = max(root.children, key=lambda c: c.visits).move
        return best_move

def play_hex(size):
    game = HexGame(size)
    maximizing_player = 'X'
    minimizing_player = '#'
    print("Maximizing Player: X")
    print("Minimizing Player: #\n")
    print("Initial Board:\n")
    start_time = time.time()

    while not game.is_game_over():
        game.print_board()
        if maximizing_player == 'X':
            move = game.get_best_move()
            print("X's move:", move)
            game.board[move[0]][move[1]] = 'X'
            maximizing_player, minimizing_player = minimizing_player, maximizing_player
        else:
            move = game.get_best_move()
            print("#'s move:", move)
            game.board[move[0]][move[1]] = '#'
            maximizing_player, minimizing_player = minimizing_player, maximizing_player

    end_time = time.time()
    game.total_elapsed_time = end_time - start_time

    game.print_board()
    if game.is_winner('#'):
        print("# wins!")
    elif game.is_winner('X'):
        print("X wins!")
    else:
        print("It's a draw!")
    print("Total Search Space:", game.total_search_space)
    print("Total Elapsed Time:", game.total_elapsed_time, "seconds")
    algorithm_name = f'MCTS {size} X {size}'
    search_space_size = game.total_search_space
    elapsed_time = game.total_elapsed_time
    append_to_csv(algorithm_name, search_space_size, elapsed_time) 
    
def append_to_csv(algorithm_name, search_space_size, elapsed_time):
    with open('hex_results.csv', 'a', newline='') as csvfile:
        fieldnames = ['Algorithm Name', 'Search Space Size', 'Elapsed Time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write header if file is empty
        if csvfile.tell() == 0:
            writer.writeheader()

        writer.writerow({'Algorithm Name': algorithm_name, 'Search Space Size': search_space_size, 'Elapsed Time': elapsed_time})

if __name__ == "__main__":
    size = int(input("Enter the size of the HEX grid: "))
    play_hex(size)


Enter the size of the HEX grid: 4
Maximizing Player: X
Minimizing Player: #

Initial Board:

- - - -
- - - -
- - - -
- - - -

X's move: (0, 2)
- - X -
- - - -
- - - -
- - - -

#'s move: (0, 1)
- # X -
- - - -
- - - -
- - - -

X's move: (0, 0)
X # X -
- - - -
- - - -
- - - -

#'s move: (3, 1)
X # X -
- - - -
- - - -
- # - -

X's move: (3, 0)
X # X -
- - - -
- - - -
X # - -

#'s move: (1, 0)
X # X -
# - - -
- - - -
X # - -

X's move: (1, 2)
X # X -
# - X -
- - - -
X # - -

#'s move: (1, 1)
X # X -
# # X -
- - - -
X # - -

X's move: (2, 1)
X # X -
# # X -
- X - -
X # - -

#'s move: (2, 2)
X # X -
# # X -
- X # -
X # - -

X's move: (2, 3)
X # X -
# # X -
- X # X
X # - -

#'s move: (3, 2)
X # X -
# # X -
- X # X
X # # -

X's move: (0, 3)
X # X X
# # X -
- X # X
X # # -

#'s move: (1, 3)
X # X X
# # X #
- X # X
X # # -

X's move: (2, 0)
X # X X
# # X #
X X # X
X # # -

#'s move: (3, 3)
X # X X
# # X #
X X # X
X # # #

It's a draw!
Total Search Space: 8024000
Total Elapsed Time: 10.1601915359